In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet 
from nltk.corpus import stopwords
import string

In [2]:
## Importing appropriate package components
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords') 
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [Errno 60] Operation
[nltk_data]     timed out>
[nltk_data] Error loading wordnet: <urlopen error [Errno 60] Operation
[nltk_data]     timed out>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 60] Operation
[nltk_data]     timed out>
[nltk_data] Error loading stopwords: <urlopen error [Errno 60]
[nltk_data]     Operation timed out>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 60] Operation timed out>


False

In [3]:
df=pd.read_json('./News_Category_Dataset_v3.json', lines=True)

In [4]:
## making columns
df=df[df['link'].duplicated()==False] ###Remove duplicate articles by link
df['headline_cleaned']=df['headline']
df['short_description_cleaned']=df['short_description']
df['headline_cleaned_withsw']=df['headline']
df['short_description_cleaned_withsw']=df['short_description']
temp_df=df
df

,link,headline,category,short_description,authors,date,headline_cleaned,short_description_cleaned,headline_cleaned_withsw,short_description_cleaned_withsw
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,Over 4 Million Americans Roll Up Sleeves For O...,Health experts said it is too early to predict...,Over 4 Million Americans Roll Up Sleeves For O...,Health experts said it is too early to predict...
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,"American Airlines Flyer Charged, Banned For Li...",He was subdued by passengers and crew when he ...,"American Airlines Flyer Charged, Banned For Li...",He was subdued by passengers and crew when he ...
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,23 Of The Funniest Tweets About Cats And Dogs ...,"""Until you have a dog you don't understand wha...",23 Of The Funniest Tweets About Cats And Dogs ...,"""Until you have a dog you don't understand wha..."
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,The Funniest Tweets From Parents This Week (Se...,"""Accidentally put grown-up toothpaste on my to...",The Funniest Tweets From Parents This Week (Se...,"""Accidentally put grown-up toothpaste on my to..."
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,Woman Who Called Cops On Black Bird-Watcher Lo...,Amy Cooper accused investment firm Franklin Te...,Woman Who Called Cops On Black Bird-Watcher Lo...,Amy Cooper accused investment firm Franklin Te...
...,...,...,...,...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,Verizon Wireless and AT&T are already promotin...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,Verizon Wireless and AT&T are already promotin...
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28,Maria Sharapova Stunned By Victoria Azarenka I...,"Afterward, Azarenka, more effusive with the pr...",Maria Sharapova Stunned By Victoria Azarenka I...,"Afterward, Azarenka, more effusive with the pr..."
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28,"Giants Over Patriots, Jets Over Colts Among M...","Leading up to Super Bowl XLVI, the most talked...","Giants Over Patriots, Jets Over Colts Among M...","Leading up to Super Bowl XLVI, the most talked..."
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28,Aldon Smith Arrested: 49ers Linebacker Busted ...,CORRECTION: An earlier version of this story i...,Aldon Smith Arrested: 49ers Linebacker Busted ...,CORRECTION: An earlier version of this story i...


In [5]:
def get_wordnet_pos(treebank_tag):
    ##### This function converts part of speech tags to a form the lemmatizer can use
    if treebank_tag.startswith('J'):
        return 'a'
    elif treebank_tag.startswith('V'):
        return 'v'
    elif treebank_tag.startswith('N'):
        return 'n'
    elif treebank_tag.startswith('R'):
        return 'r'
    else:
        return 'n'

In [6]:
def preprocess_text(tstring):
    temp=''
    
    ## 1st we tokenize the words from the headlines
    tlist=nltk.word_tokenize(tstring.translate(str.maketrans('-', ' ', '—\'\"‘')))
                             
    ## 2nd we convert to lowercase and remove punctation
    tlist =[w.lower() for w in tlist if (w not in string.punctuation) and (w not in ['\u2019', 's'])]
    
    tlist=nltk.pos_tag(tlist) ##### Tag part of speech
    
    ## Then we remove stop words
    tlist=[w for w in tlist if not w[0] in set(stopwords.words('english'))]

    for word in tlist:
        #print(word[0], get_wordnet_pos(word[1]))
        if len(temp)!=0:
            #print(WordNetLemmatizer().lemmatize(word[0].lower(),get_wordnet_pos(word[1])))
            temp += ' ' + WordNetLemmatizer().lemmatize(word[0].lower(),get_wordnet_pos(word[1]))
        else: 
            #print(WordNetLemmatizer().lemmatize(word[0].lower(),get_wordnet_pos(word[1])))
            temp += WordNetLemmatizer().lemmatize(word[0].lower(),get_wordnet_pos(word[1]))
    return temp

In [8]:
#####Displaying sample output
for i in range(min(df.shape[0], 300)):
    temp=preprocess_text(df['headline_cleaned'][i])
    print(df['headline_cleaned'].iat[i])
    print(temp) # This prints it in sting format 


Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters
4 million american roll sleeve omicron target covid booster
American Airlines Flyer Charged, Banned For Life After Punching Flight Attendant On Video
american airline flyer charge ban life punch flight attendant video
23 Of The Funniest Tweets About Cats And Dogs This Week (Sept. 17-23)
23 funny tweet cat dog week sept. 17 23
The Funniest Tweets From Parents This Week (Sept. 17-23)
funny tweet parent week sept. 17 23
Woman Who Called Cops On Black Bird-Watcher Loses Lawsuit Against Ex-Employer
woman call cop black bird watcher lose lawsuit ex employer
Cleaner Was Dead In Belk Bathroom For 4 Days Before Body Found: Police
cleaner dead belk bathroom 4 day body find police
Reporter Gets Adorable Surprise From Her Boyfriend While Live On TV
reporter get adorable surprise boyfriend live tv
Puerto Ricans Desperate For Water After Hurricane Fiona’s Rampage
puerto ricans desperate water hurricane fiona rampage
How A N

Putin Plans To Snub Gorbachev's Funeral
putin plan snub gorbachevs funeral
NASA Aims For Saturday Launch Of New Moon Rocket After Fixing Fuel Leaks
nasa aim saturday launch new moon rocket fix fuel leak
U.S. Hiring Slowed In August As Employers Add 315,000 Jobs
u.s hiring slow august employer add 315,000 job
Pregnant Woman Who Claimed Unborn Baby Was HOV Lane Passenger Receives Another Ticket
pregnant woman claim unborn baby hov lane passenger receive another ticket
Should You Freeze-Dry Your Breast Milk?
freeze dry breast milk
Missing Louisiana Nun Found 'Alive And Safe' Months After Armed Home Raid
miss louisiana nun find alive safe month arm home raid
A Ku Klux Klan Plaque Is Mounted At West Point's Science Center, Commission Finds
ku klux klan plaque mount west point science center commission find
Toxic Algae Causes Thousands Of Dead Fish To Wash Ashore In California
toxic algae cause thousand dead fish wash ashore california
Lisa Loeb Paved Her Own Way In The '90s — A Rare Feat Fo

U.S. Unemployment Claims Rise To Highest Level Since November
u.s. unemployment claim rise high level since november
Gas Prices Dip Just Below $4 For The First Time In 5 Months
gas price dip 4 first time 5 month
As Joe Biden Wins Mount, 2024 Doubts Ease For Top House Dems
joe biden win mount 2024 doubt ease top house dems
FX's 'Justified' Set Faces Violent Interruption For Second Time In 3 Weeks
fxs justified set face violent interruption second time 3 week
Ex-Twitter Staffer Convicted For Sharing Private Information On Dissidents With Saudis
ex twitter staffer convict share private information dissident saudi
Stranded Whale Euthanized After Removal From French River
strand whale euthanized removal french river
Sesame Place To Train Workers On Diversity After Lawsuit
sesame place train worker diversity lawsuit
Kobe Bryant Crash Photos Lawsuit To Be Heard By LA Jury
kobe bryant crash photo lawsuit hear la jury
Why The Justice Department Can’t Say More About The Mar-A-Lago Raid
justice d

In [11]:
###Apply the preprocessing fucntion to the headline and short_description columns of the dataset
%%time
df['short_description_cleaned']=temp_df['short_description_cleaned'].apply(lambda x: preprocess_text(x))

CPU times: user 16min 25s, sys: 3min 10s, total: 19min 36s
Wall time: 20min 25s


In [12]:
%%time
df['headline_cleaned']=temp_df['headline_cleaned'].apply(lambda x: preprocess_text(x))

CPU times: user 8min 31s, sys: 1min 34s, total: 10min 6s
Wall time: 10min 20s


In [15]:
df

,link,headline,category,short_description,authors,date,headline_cleaned,short_description_cleaned,headline_cleaned_withsw,short_description_cleaned_withsw
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,4 million american roll sleeve omicron target ...,health expert say early predict whether demand...,over 4 million american roll up sleeve for omi...,health expert say it be too early to predict w...
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,american airline flyer charge ban life punch f...,subdue passenger crew flee back aircraft confr...,american airline flyer charge ban for life aft...,he be subdue by passenger and crew when he fle...
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,23 funny tweet cat dog week sept. 17 23,dog dont understand could eat,23 of the funny tweet about cat and dog this w...,until you have a dog you dont understand what ...
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,funny tweet parent week sept. 17 23,accidentally put grow toothpaste toddler tooth...,the funny tweet from parent this week sept. 17 23,accidentally put grow up toothpaste on my todd...
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,woman call cop black bird watcher lose lawsuit...,amy cooper accuse investment firm franklin tem...,woman who call cop on black bird watcher lose ...,amy cooper accuse investment firm franklin tem...
...,...,...,...,...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28,rim ceo thorsten heins significant plan blackb...,verizon wireless already promote lte device in...,rim ceo thorsten heins significant plan for bl...,verizon wireless and at t be already promote l...
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28,maria sharapova stun victoria azarenka austral...,afterward azarenka effusive press normal credi...,maria sharapova stun by victoria azarenka in a...,afterward azarenka more effusive with the pres...
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28,giant patriots jet colt among improbable super...,lead super bowl xlvi talked game could end one...,giant over patriots jet over colt among most i...,lead up to super bowl xlvi the most talked abo...
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28,aldon smith arrest 49ers linebacker bust duo,correction early version story incorrectly sta...,aldon smith arrest 49ers linebacker bust for duo,correction an early version of this story inco...


In [9]:
# Save output to a pickle file
df.to_pickle('./News_ds_cleaned.pkl') 